# 1. Разобраться как использовать мультииндексные ключи в данном примере

In [2]:
import numpy as np
import pandas as pd

In [8]:
index = [
    ('city_1', 2010),
    ('city_1', 2020),
    ('city_2', 2010),
    ('city_2', 2020),
    ('city_3', 2010),
    ('city_3', 2020),
]

population = [
    101,
    201,
    102,
    202,
    103,
    203,
]

pop = pd.Series(population, index = index)
pop_df = pd.DataFrame(
    {
        'total': pop,
        'something': [
            10,
            11,
            12,
            13,
            14,
            15,
        ]
    }
)

i = pd.MultiIndex.from_tuples(index)
pop_df = pop_df.reindex(i)
pop_df

total  something
city_1 2010    101         10
       2020    201         11
city_2 2010    102         12
       2020    202         13
city_3 2010    103         14
       2020    203         15

In [11]:
pop_df_1 = pop_df.loc['city_1', 'something']
pop_df_1

2010    10
2020    11
Name: something, dtype: int64

In [13]:
pop_df_1 = pop_df.loc[['city_1', 'city_3'], ['total', 'something']]
pop_df_1

total  something
city_1 2010    101         10
       2020    201         11
city_3 2010    103         14
       2020    203         15

In [14]:
pop_df_1 = pop_df.loc[['city_1', 'city_3'], 'something']
pop_df_1

city_1  2010    10
        2020    11
city_3  2010    14
        2020    15
Name: something, dtype: int64

# 2. Из получившихся данных выбрать данные по 
# - 2020 году (для всех столбцов)
# - job_1 (для всех строк)
# - для city_1 и job_2 

In [18]:
index = pd.MultiIndex.from_product(
    [
        ["city_1", "city_2"],
        ["2010", "2020"]
    ],
    names=["city", "year"]
)

columns = pd.MultiIndex.from_product(
    [
        ["person_1", "person_2", "person_3"],
        ["job_1", "job_2"]
    ],
    names=["worker", "job"]
)

rng = np.random.default_rng(1)
data = rng.random((4, 6))

data_df = pd.DataFrame(data, index=index, columns=columns)
data_df

worker       person_1            person_2            person_3          
job             job_1     job_2     job_1     job_2     job_1     job_2
city   year                                                            
city_1 2010  0.511822  0.950464  0.144160  0.948649  0.311831  0.423326
       2020  0.827703  0.409199  0.549594  0.027559  0.753513  0.538143
city_2 2010  0.329732  0.788429  0.303195  0.453498  0.134042  0.403113
       2020  0.203455  0.262313  0.750365  0.280409  0.485191  0.980737

In [26]:
data_df_1 = data_df.xs("2020", level="year")
data_df_1

worker  person_1            person_2            person_3          
job        job_1     job_2     job_1     job_2     job_1     job_2
city                                                              
city_1  0.827703  0.409199  0.549594  0.027559  0.753513  0.538143
city_2  0.203455  0.262313  0.750365  0.280409  0.485191  0.980737

In [30]:
data_df_1 = data_df.xs("job_1", level="job", axis=1)
data_df_1

worker       person_1  person_2  person_3
city   year                              
city_1 2010  0.511822  0.144160  0.311831
       2020  0.827703  0.549594  0.753513
city_2 2010  0.329732  0.303195  0.134042
       2020  0.203455  0.750365  0.485191

In [35]:
data_df_city = data_df.xs("city_1", level="city")
data_df_1 = data_df_city.xs("job_2", level="job", axis=1)
data_df_1

worker,person_1,person_2,person_3
year,,,
2010,0.950464,0.948649,0.423326
2020,0.409199,0.027559,0.538143


# 3. Выполнить запрос на получение следующих данных
# - все данные по person_1 и person_3
# - все данные по первому городу и первым двум person-ам (с использование срезов)

In [37]:
data_df_1 = data_df.loc[:, ["person_1", "person_3"]]
data_df_1

worker       person_1            person_3          
job             job_1     job_2     job_1     job_2
city   year                                        
city_1 2010  0.511822  0.950464  0.311831  0.423326
       2020  0.827703  0.409199  0.753513  0.538143
city_2 2010  0.329732  0.788429  0.134042  0.403113
       2020  0.203455  0.262313  0.485191  0.980737

In [42]:
data_df_1 = data_df.loc["city_1", pd.IndexSlice["person_1":"person_2"]]
data_df_1

worker  person_1            person_2          
job        job_1     job_2     job_1     job_2
year                                          
2010    0.511822  0.950464  0.144160  0.948649
2020    0.827703  0.409199  0.549594  0.027559

In [43]:
data_df_1 = data_df.loc[pd.IndexSlice[:, "2020"], pd.IndexSlice["person_1":"person_2"]]
data_df_1

worker       person_1            person_2          
job             job_1     job_2     job_1     job_2
city   year                                        
city_1 2020  0.827703  0.409199  0.549594  0.027559
city_2 2020  0.203455  0.262313  0.750365  0.280409

# 4. Привести пример использования inner и outer джойнов для Series

In [3]:
ser1 = pd.Series(['a', 'b', 'c'], index=[1, 2, 3])
ser2 = pd.Series(['b', 'c', 'f'], index=[1, 2, 6])

print(pd.concat([ser1, ser2], axis=1, join="outer"))
print(pd.concat([ser1, ser2], axis=1, join="inner"))

     0    1
1    a    b
2    b    c
3    c  NaN
6  NaN    f
   0  1
1  a  b
2  b  c
